In [ ]:
# Importing necessary packages:

from glob import glob
import numpy as np
import pandas as pd

import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage import organizer
from polyphys.analyze import measurer
from polyphys.manage import utilizer

# Project details

project_details ={
    'SumRule':{
        'parser': SumRule,
        'space_pat': 'N*D*ac*',
        'hierarchy': 'N*',
        'space_hierarchy': 'N*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon', 'dcyl',
                       'dcrowd','phi_c_bulk'
                      ],
        'time_varying_props': [ 'asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'rfloryTMon','shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['space', 'ensemble_long', 'ensemble', 'nmon',
                             'dcyl','dcrowd', 'phi_c_bulk',
                             'phi_c_bulk_round'
                            ],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'rfloryMon-mean',
                             'rfloryMon-var', 'rfloryMon-sem',
                             'shapeMon-mean', 'shapeMon-var', 'shapeMon-sem']
    },
    'TransFoci':{
        'parser': TransFoci,
        'space_pat': 'ns*nl*al*D*ac*',
        'hierarchy': 'eps*',
        'space_hierarchy': 'ns*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon_small',
                       'nmon_large','dmon_large', 'dcyl', 'dcrowd',
                       'phi_c_bulk'
                      ],
        'time_varying_props': ['asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['ensemble_long', 'ensemble', 'space', 'dcyl',
                             'dmon_large', 'nmon_large', 'nmon_small',
                             'dcrowd', 'phi_c_bulk', 'phi_c_bulk_round'],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'shapeMon-mean',
                             'shapeMon-var', 'shapeMon-sem']
    }
}

norm_props_specs =  {
    'SumRule': {
        'rfloryMon-norm': {
            'name': 'Flory radius',
            'symbol': r'$\frac{R_F(\phi_c)}{R_{F,0}}$',
            'color': 'firebrick'
        },
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{\fract{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        }
    },
    'TransFoci':{
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        },
        'bondsHistFoci-norm': {
            'name': 'Probability distribution function of direct bonds',
            'symbol': r'$\mathcal{P}(x_b)$',
            'color': 'orchid'
        },
        'clustersHistFoci-norm': {
            'name': 'Probability distribution function of clusters',
            'symbol': r'$\mathcal{P}(x_c)$',
            'color': 'orchid'
        }
    }
}

attr_labels = {
    "phi_c_bulk_round": "$\phi_c$",
    "time": "$\hat{{t}}$",
    "lags": "$\hat{t}_{lags}$",
    "dmon_large": "$a_l$",
    "dcrowd": "$a_c$",
    "dcyl": "$D$",
    "nmon_small": "$n_s$",
    "nmon_large": "$n_l$",
    'phi_c_bulk_norm': "${a\phi_c}/{a_c}$"
}

# loading databases:
analysis_db = '/Users/amirhsi_mini/research_data/analysis/'
project = 'TransFoci'
phase='ensAvg'
geometry='biaxial'
group = "bug"
species = "Mon"
space_dbs = glob(analysis_db + project_details[project]['space_pat'])

In [ ]:
species = 'Foci'
pair_dist_properties = ['pairDistHistFoci', 'pairDistRdfFoci']
prop = pair_dist_properties[1]
bin_center_labels = {
   'pairDistHistFoci': '$r$',
   'pairDistRdfFoci': '$r$' 
}
attr_labels['bin_center'] = bin_center_labels[prop]
# LOad data
dist_map = '-'.join([project, group, species, 'pairDistStats.parquet.brotli'])
dist_map = analysis_db + dist_map
dist_map = pd.read_parquet(dist_map)
# a sorted list of unique spaces in the dataset
spaces = dist_map['space'].unique()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
dist_map['phi_c_bulk_round'] = dist_map['phi_c_bulk'].apply(
        utilizer.round_up_nearest, args=[divisor, round_to]
        )
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
dist_map = dist_map.loc[~dist_map['phi_c_bulk_round'].isin(phi_c_to_drop),:]
dist_map.reset_index(inplace=True, drop=True)
# Dropping pair distance cols of `prop_to_drop`
prop_to_drop = pair_dist_properties[0]
cols_to_drop = [col for col in dist_map.columns if col.startswith(prop_to_drop)]
dist_map.drop(columns=cols_to_drop, inplace=True)
# renaming pair distance cols of `prop`
pair_tags = {
    col: col.split('-')[1] for col in dist_map.columns if 'genDist' in col
    }
dist_map.rename(columns=pair_tags, inplace=True)
# Now, just keep the new col names for melting
pair_tags = list(pair_tags.values())
pair_names = [organizer.sort_by_alphanumeric(tag)[1::2] for tag in pair_tags]
pair_names = [
    r"$\vec{{n}}_1={0}, \vec{{n}}_2={1}, \Delta\vec{{n}}={2}$".format(int(locs[0]), int(locs[1]), int(locs[2])) for locs in pair_names]
pair_names
attributes = project_details[project]['equil_attributes'].copy()
attributes.extend(['bin_center', 'bin_center_norm'])
sel_cols = attributes + pair_tags
dist_map_melted = dist_map[sel_cols].melt(
    id_vars=attributes,
    value_vars=pair_tags,
    var_name=prop
)

In [ ]:
heat_m_cols = pair_tags.copy()
heat_m_cols.extend(['bin_center'])
print(heat_m_cols)
phi_c_unique = dist_map['phi_c_bulk_round'].unique()
dmon_l_unique = dist_map['dmon_large'].unique()

for phi_c in phi_c_unique[:1]:
    for dmon_large in dmon_l_unique[:1]:
        phi_cond =(dist_map['phi_c_bulk_round']==phi_c)
        dmon_cond = (dist_map['dmon_large']==dmon_large)
        rdfs = dist_map.loc[phi_cond & dmon_cond, heat_m_cols]
        rdfs.set_index('bin_center', inplace=True)
rdfs = rdfs.T
ax = sns.heatmap(rdfs, cmap='rocket_r', robust=True, yticklabels=pair_names, xticklabels=False, cbar=False)
ax.set_xlabel(attr_labels['bin_center'])
ax.set_ylabel("Pair genomic position and distance")

In [ ]:
sel_attrs = ['bin_center', 'dmon_large', 'phi_c_bulk_round']
sel_cols = pair_tags + sel_attrs 
phi_c_chosen = [0., 0.2, 0.25, 0.3, 0.35, 0.4]
rdfs = dist_map.loc[dist_map['phi_c_bulk_round'].isin(phi_c_chosen), sel_cols]
rdfs.set_index(['bin_center'], inplace=True)
rdfs

In [ ]:
def draw_heatmap(pair_tags, **kwargs):
    data = kwargs.pop('data')
    data_subset = data.loc[:,pair_tags]
    data_T = data_subset.T
    sns.heatmap(data_T, **kwargs)

fg = sns.FacetGrid(rdfs, col='dmon_large', row='phi_c_bulk_round')
fg.map_dataframe(draw_heatmap, pair_tags, cmap='rocket_r', robust=True, yticklabels=pair_names, xticklabels=False, cbar=False)
plt.show()

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# Generate a set of sample data
np.random.seed(0)
indices = pd.MultiIndex.from_product((range(5), range(5), range(5)),
 names=('label0', 'label1', 'label2'))
data = pd.DataFrame(np.random.uniform(0, 100, size=len(indices)),
 index=indices, columns=('value',)).reset_index()

def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    sns.heatmap(d, **kwargs)

fg = sns.FacetGrid(data, col='label0')
fg.map_dataframe(draw_heatmap, 'label1', 'label2', 'value', cbar=False, square = True)
# get figure background color
facecolor=plt.gcf().get_facecolor()
for ax in fg.axes.flat:
    # set aspect of all axis
    ax.set_aspect('equal','box-forced')
    # set background color of axis instance
    ax.set_axis_bgcolor(facecolor)
plt.show()